# Heisenberg Model

In [8]:
using LinearAlgebra
using TensorKit
using MPSKit
using MPSKitModels
using WignerSymbols

## 1. MPSKitModels:

In [13]:
S_auto = S_exchange(ComplexF64, SU2Irrep; spin=1);

In [40]:
lattice = InfiniteChain(1)
H_auto = heisenberg_XXX(ComplexF64, SU2Irrep, lattice; J=1.0, spin=1);
SS1 = S_exchange(ComplexF64, SU2Irrep; spin=1);

In [ ]:
function S_exchange(elt::Type{<:Number}, ::Type{SU2Irrep}; spin=1 // 2)
    pspace = SU2Space(spin => 1)
    aspace = SU2Space(1 => 1)

    Sleft = TensorMap(ones, elt, pspace ← pspace ⊗ aspace)
    Sright = -TensorMap(ones, elt, aspace ⊗ pspace ← pspace)

    @tensor SS[-1 -2; -3 -4] := Sleft[-1; -3 1] * Sright[1 -2; -4] * (spin^2 + spin)
    return SS
end

function heisenberg_XXX end
function heisenberg_XXX(lattice::AbstractLattice; kwargs...)
    return heisenberg_XXX(ComplexF64, Trivial, lattice; kwargs...)
end
function heisenberg_XXX(symmetry::Type{<:Sector}, lattice::AbstractLattice=InfiniteChain(1);
                        kwargs...)
    return heisenberg_XXX(ComplexF64, symmetry, lattice; kwargs...)
end
function heisenberg_XXX(elt::Type{<:Number}, lattice::AbstractLattice; kwargs...)
    return heisenberg_XXX(elt, Trivial, lattice; kwargs...)
end
function heisenberg_XXX(T::Type{<:Number}=ComplexF64,
                        symmetry::Type{<:Sector}=Trivial,
                        lattice::AbstractLattice=InfiniteChain(1);
                        J::Real=1.0, spin::Real=1)
    term = rmul!(S_exchange(T, symmetry; spin=spin), J)
    return @mpoham sum(nearest_neighbours(lattice)) do (i, j)
        return term{i,j}
    end
end

##  2. quadratic Casimir:

In [38]:
casimir(s::SU2Irrep) = s.j * (s.j + 1)

function heisenberg_hamiltonian(; J=-1.0)
    s = SU2Irrep(1)
    ℋ = SU2Space(1 => 1)
    SS = TensorMap(zeros, ComplexF64, ℋ ⊗ ℋ ← ℋ ⊗ ℋ)
    for (S, data) in blocks(SS)
        data .= -0.5J * (casimir(S) - casimir(s) - casimir(s))
    end
    return SS, MPOHamiltonian(SS)
end
SS2 = heisenberg_hamiltonian()[1]
H_qC = heisenberg_hamiltonian()[2];

## 3. Wigner-Eckart Theorem:

In [13]:
Sx = 1 / sqrt(2) * ComplexF64[0 1 0; 1 0 1; 0 1 0]
Sy = 1 / sqrt(2) * ComplexF64[0 1im 0; -1im 0 1im; 0 -1im 0]
Sz = ComplexF64[-1 0 0; 0 0 0; 0 0 1]

@tensor SS_arr[-1 -2; -3 -4] := Sx[-1; -3] * Sx[-2; -4] + Sy[-1; -3] * Sy[-2; -4] + Sz[-1; -3] * Sz[-2; -4];

In [14]:
function get_reduced_element(k)
    # construct Clebsch-Gordan coefficients for coupling 1 ⊗ 1 to k   
    CG = zeros(ComplexF64, 3, 3, 2*k + 1)
    for m1 in -k:k, m2 in -1:1, m3 in -1:1
        CG[m2 + 2, m3 + 2, m1 + k + 1] = clebschgordan(1, m2, 1, m3, k, m1)
    end

    # project out diagonal matrix on coupled irrep space
    @tensor reduced_matrix[-1; -2] := CG[1 2; -1] * SS_arr[1 2; 3 4] * conj(CG[3 4; -2])

    # check that it is proportional to the identity
    @assert isapprox(reduced_matrix, reduced_matrix[1, 1] * I; atol=1e-12)

    # return the proportionality factor
    return reduced_matrix[1, 1]
end

V = SU2Space(1 => 1)
SS = TensorMap(zeros, ComplexF64, V ⊗ V ← V ⊗ V)
for (s, f) in fusiontrees(SS)
    k = Int(f.coupled.j)
    SS[s, f] .= get_reduced_element(k)
end
SS3 = SS;

In [42]:
lattice = InfiniteChain(1)
H_WE = @mpoham begin
    sum(nearest_neighbours(lattice)) do (i, j)
        return SS{i,j}
    end
end;

## Check

In [43]:
SS1 ≈ SS2 ≈ SS3

true

In [20]:
V = SU2Space(1 => 1)
SS4 = TensorMap(zeros, ComplexF64, V ⊗ V ← V ⊗ V)
blocks(SS4)[SU2Irrep(0)][1,1] = -2
blocks(SS4)[SU2Irrep(1)][1,1] = -1
blocks(SS4)[SU2Irrep(2)][1,1] = 1
SS4 ≈ SS3

true

In [37]:
H_qC ≈ H_auto ≈ H_WE

true

In [18]:
H = heisenberg_XXX(ComplexF64, SU2Irrep, InfiniteChain(2); spin=1//2);

In [19]:
P = Rep[SU₂](1//2 => 1)
V1 = Rep[SU₂](1//2 => 50, 3//2 => 25)
V2 = Rep[SU₂](0 => 80, 1 => 80, 2=> 40)
state = InfiniteMPS([P, P], [V1, V2]);

In [20]:
groundstate, cache, delta = find_groundstate(state, H, VUMPS(; maxiter=400, tol_galerkin=1e-12));

┌ Info: VUMPS iteration:
│   iter = 1
│   ϵ = 0.08893768627027673
│   λ = -0.8821389040993324 - 2.378501115724251e-16im
│   Δt = 0.649920792
└ @ MPSKit /Users/zongyy/.julia/packages/MPSKit/atykv/src/algorithms/groundstate/vumps.jl:90
┌ Info: VUMPS iteration:
│   iter = 2
│   ϵ = 0.009226516319418879
│   λ = -0.8858077421906676 + 6.602946740391882e-17im
│   Δt = 0.658653417
└ @ MPSKit /Users/zongyy/.julia/packages/MPSKit/atykv/src/algorithms/groundstate/vumps.jl:90
┌ Info: VUMPS iteration:
│   iter = 3
│   ϵ = 0.0027106568266109196
│   λ = -0.8861962494427349 - 2.0235565493835012e-16im
│   Δt = 0.594616375
└ @ MPSKit /Users/zongyy/.julia/packages/MPSKit/atykv/src/algorithms/groundstate/vumps.jl:90
┌ Info: VUMPS iteration:
│   iter = 4
│   ϵ = 0.0010954123441341173
│   λ = -0.8862647364824582 - 6.845349702794832e-17im
│   Δt = 0.624735917
└ @ MPSKit /Users/zongyy/.julia/packages/MPSKit/atykv/src/algorithms/groundstate/vumps.jl:90
┌ Info: VUMPS iteration:
│   iter = 5
│   ϵ = 0.0004716933

In [21]:
expectation_value(groundstate, H, cache)#1/4 - log(2)

2-element PeriodicArray{ComplexF64, 1}:
 -0.42769697369193055 + 7.752045533271357e-18im
  -0.4585864500165841 - 5.816178202101255e-17im

In [22]:
groundstate, cache, delta = find_groundstate(state, H, IDMRG2(trscheme=truncbelow(1e-5)));

┌ Info: idmrg iter 1 err 0.01616776517774077
└ @ MPSKit /Users/zongyy/.julia/packages/MPSKit/atykv/src/algorithms/groundstate/idmrg.jl:178
┌ Info: idmrg iter 2 err 0.011924192785993446
└ @ MPSKit /Users/zongyy/.julia/packages/MPSKit/atykv/src/algorithms/groundstate/idmrg.jl:178
┌ Info: idmrg iter 3 err 0.00923874607320577
└ @ MPSKit /Users/zongyy/.julia/packages/MPSKit/atykv/src/algorithms/groundstate/idmrg.jl:178
┌ Info: idmrg iter 4 err 0.007498996789628296
└ @ MPSKit /Users/zongyy/.julia/packages/MPSKit/atykv/src/algorithms/groundstate/idmrg.jl:178
┌ Info: idmrg iter 5 err 0.006293365648630694
└ @ MPSKit /Users/zongyy/.julia/packages/MPSKit/atykv/src/algorithms/groundstate/idmrg.jl:178
┌ Info: idmrg iter 6 err 0.00541207269091504
└ @ MPSKit /Users/zongyy/.julia/packages/MPSKit/atykv/src/algorithms/groundstate/idmrg.jl:178
┌ Info: idmrg iter 7 err 0.004741299617594525
└ @ MPSKit /Users/zongyy/.julia/packages/MPSKit/atykv/src/algorithms/groundstate/idmrg.jl:178
┌ Info: idmrg iter 8 er

In [23]:
expectation_value(groundstate, H, cache)

2-element PeriodicArray{ComplexF64, 1}:
 -0.4359499148565782 - 1.0447799605332413e-18im
 -0.4503440875385665 - 4.990152404381441e-18im

In [ ]:
H = heisenberg_XXX(ComplexF64, SU2Irrep, FiniteChain(12); spin=1//2);
P = Rep[SU₂](1//2 => 1)
V1 = Rep[SU₂](1//2 => 50, 3//2 => 25)
V2 = Rep[SU₂](0 => 80, 1 => 80, 2=> 40)
state = FiniteMPS([P, P], [V1, V2]);
groundstate, cache, delta = find_groundstate(state, H, VUMPS(; maxiter=400, tol_galerkin=1e-12));